In [65]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [59]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)"
page = requests.get(url)
soup = BeautifulSoup(page.text, "html")

In [60]:
table = soup.find('table')


In [85]:
# finding the column
head = table.find_all('th')
# make them into a list
lst = [re.sub(r'\[.*?\]', '', h.text.strip()) for h in head]
df = pd.DataFrame(columns=lst)
df


,Location,UN Continental Region,UN Statistical Subregion,Population(1 July 2022),Population(5 March 2024),Change


In [86]:
table.find_all('tr')[3]

<tr>
<td><span class="datasortkey" data-sort-value="China"><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_China.svg.png 2x" width="23"/></span></span> </span><a href="/wiki/China" title="China">China</a></span><sup class="reference" id="cite_ref-7"><a href="#cite_note-7">[a]</a></sup></td>
<td><a href="/wiki/Asia" title="Asia">Asia</a></td>
<td><a href="/wiki/East_Asia" title="East Asia">Eastern Asi

In [87]:
# finding the row data
cols = table.find_all('tr')
for row in cols[2:]:
    row_data = row.find_all('td')
    country_tag = row.find('a')
    country_name = [country_tag.text]
    row_lst = [ind.text.strip() for ind in row_data]
    length = len(df)
    df.loc[length] = row_lst
df


,Location,UN Continental Region,UN Statistical Subregion,Population(1 July 2022),Population(5 March 2024),Change
0,India,Asia,Southern Asia,"1,417,173,173","1,428,627,663",+0.81%
1,China[a],Asia,Eastern Asia,"1,425,887,337","1,425,671,352",−0.02%
2,United States,Americas,Northern America,"338,289,857","339,996,564",+0.50%
3,Indonesia,Asia,South-eastern Asia,"275,501,339","277,534,123",+0.74%
4,Pakistan,Asia,Southern Asia,"235,824,862","240,485,658",+1.98%
...,...,...,...,...,...,...
233,Falkland Islands (United Kingdom),Americas,South America,"3,780","3,791",+0.29%
234,Niue,Oceania,Polynesia,"1,934","1,935",+0.05%
235,Tokelau (New Zealand),Oceania,Polynesia,"1,871","1,893",+1.18%
236,Vatican City[x],Europe,Southern Europe,510,518,N/A


In [90]:
for idx, row in enumerate(df["Location"]):
    # Apply re.sub() to remove [x] from each value
    df["Location"][idx] = re.sub(r'\[.*?\]', '', row)


array(['Asia', 'Americas', 'Africa', 'Europe', 'Oceania'], dtype=object)

In [99]:
#find average percentage change in the world
# if N/A assume no change
df["Change"] = df["Change"].replace("N/A", "0.00")
for idx, row in enumerate(df["Change"]):
    df["Change"][idx] = re.sub(r'%', '', row)
    if (re.match(r'+[0-9]*\.?[0-9]+')):
        df["Change"][idx] = re.sub(r'+', '', row)
df


,Location,UN Continental Region,UN Statistical Subregion,Population(1 July 2022),Population(5 March 2024),Change
0,India,Asia,Southern Asia,"1,417,173,173","1,428,627,663",+0.81
1,China,Asia,Eastern Asia,"1,425,887,337","1,425,671,352",−0.02
2,United States,Americas,Northern America,"338,289,857","339,996,564",+0.50
3,Indonesia,Asia,South-eastern Asia,"275,501,339","277,534,123",+0.74
4,Pakistan,Asia,Southern Asia,"235,824,862","240,485,658",+1.98
...,...,...,...,...,...,...
233,Falkland Islands (United Kingdom),Americas,South America,"3,780","3,791",+0.29
234,Niue,Oceania,Polynesia,"1,934","1,935",+0.05
235,Tokelau (New Zealand),Oceania,Polynesia,"1,871","1,893",+1.18
236,Vatican City,Europe,Southern Europe,510,518,0
